In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading the data I scraped from weltfussball.de:
weltfussball = pd.read_csv('weltfussball_data.csv')

In [3]:
# Replacing league names by those used by 538:
weltfussball['league'] = weltfussball.league.map({
                         'Ligue 1': 'French Ligue 1',
                         'Primera División': 'Spanish Primera Division',
                         'Serie A': 'Italy Serie A',
                         'Bundesliga': 'German Bundesliga',
                         'Primera División 2016/201': 'Mexican Primera Division Torneo Clausura',
                         'Primera División 2017/201': 'Mexican Primera Division Torneo Clausura',
                         'Primera División 2018/201': 'Mexican Primera Division Torneo Clausura',
                         'Primera División 2019/202': 'Mexican Primera Division Torneo Clausura',
                         'Sé': 'Brasileiro Série A',
                         '2. Bundesliga': 'German 2. Bundesliga',
                         'Championship': 'English League Championship',
                         'Primeira Liga': 'Portuguese Liga',
                         'Super L': 'Chinese Super League',
                         'A-League': 'Australian A-League',
                         'Serie B': 'Italy Serie B'})

In [4]:
# Filling in the league name of the English Premier League:
for i in weltfussball.index:
    if i >= 1419 and i <= 2846:
        weltfussball.league.fillna('Barclays Premier League', limit=1, inplace=True)

In [5]:
# Loading the dataset from 538 and selecting only those matches for which 538 provides xG scores:
spi = pd.read_csv('spi_matches.csv')
spi = spi[spi.xg1.notnull()].reset_index(drop=True)

In [6]:
# Renaming columns in the weltfussball frame:
weltfussball.rename(columns={'home_team': 'team1', 'away_team': 'team2', 'home_score': 'score1', 'away_score': 'score2'}, inplace=True)
weltfussball.reset_index(inplace=True)

In [7]:
# Creating a dictionary that maps the names of the teams in weltfussball to the names of these teams in spi:
map_dict = {}
for k in weltfussball.index:
    if weltfussball.team1.iloc[k] in map_dict.keys() and weltfussball.team2.iloc[k] in map_dict.keys():
        continue
    for i in spi.index:
        if spi.league.iloc[i] == weltfussball.league.iloc[k] and spi.date.iloc[i] == weltfussball.date.iloc[k]:
            if spi.team1.iloc[i] == weltfussball.team1.iloc[k] or spi.team2.iloc[i] == weltfussball.team2.iloc[k]:
                map_dict[weltfussball.team1.iloc[k]] = spi.team1.iloc[i]
                map_dict[weltfussball.team2.iloc[k]] = spi.team2.iloc[i]
                break
            elif spi.score1.iloc[i] == weltfussball.score1.iloc[k] and spi.score2.iloc[i] == weltfussball.score2.iloc[k]:
                double = False
                for j in spi.index:
                    if i == j:
                        continue
                    elif spi.league.iloc[j] == weltfussball.league.iloc[k] and spi.date.iloc[j] == weltfussball.date.iloc[k]:
                        if spi.score1.iloc[j] == weltfussball.score1.iloc[k] and spi.score2.iloc[j] == weltfussball.score2.iloc[k]:
                            double = True
                            break
                if double == False:
                    map_dict[weltfussball.team1.iloc[k]] = spi.team1.iloc[i]
                    map_dict[weltfussball.team2.iloc[k]] = spi.team2.iloc[i]
                    break

In [8]:
# Displaying the dictionary:
map_dict

{'SC Bastia': 'Bastia',
 'Paris Saint-Germain': 'Paris Saint-Germain',
 'AS Monaco': 'AS Monaco',
 'EA Guingamp': 'Guingamp',
 'Dijon FCO': 'Dijon FCO',
 'FC Nantes': 'Nantes',
 'Montpellier HSC': 'Montpellier',
 'Angers SCO': 'Angers',
 'AS Nancy': 'AS Nancy Lorraine',
 'Olympique Lyon': 'Lyon',
 'OGC Nice': 'Nice',
 'Stade Rennes': 'Stade Rennes',
 'Olympique Marseille': 'Marseille',
 'Toulouse FC': 'Toulouse',
 'SM Caen': 'Caen',
 'Lille OSC': 'Lille',
 'FC Lorient': 'Lorient',
 'Girondins Bordeaux': 'Bordeaux',
 'AS Saint-Étienne': 'St Etienne',
 'FC Metz': 'Metz',
 'Amiens SC': 'Amiens',
 'ESTAC Troyes': 'Troyes',
 'RC Strasbourg': 'Strasbourg',
 'Nîmes Olympique': 'Nimes',
 'Stade Reims': 'Reims',
 'Stade Brest': 'Brest',
 'Hull City': 'Hull City',
 'Leicester City': 'Leicester City',
 'Burnley FC': 'Burnley',
 'Swansea City': 'Swansea City',
 'Crystal Palace': 'Crystal Palace',
 'West Bromwich Albion': 'West Bromwich Albion',
 'Everton FC': 'Everton',
 'Tottenham Hotspur': 'Tott

In [9]:
# Using the dictionary to replace the teams' names in weltfussball by those used by 538:
weltfussball['team1'] = weltfussball.team1.map(map_dict)
weltfussball['team2'] = weltfussball.team2.map(map_dict)

In [10]:
# Adding the data from weltfussball.de to the spi frame by performing a left merge:
spi = spi.merge(weltfussball, on = ['team1', 'team2', 'score1', 'score2', 'date', 'league'], how='left')

In [11]:
# Counting how many matches have been provided with data from weltfussball.de:
spi.draw_time.notnull().value_counts() # not perfect, but good enough for me

True     10878
False     5804
Name: draw_time, dtype: int64

In [12]:
# Saving the merged data frame:
spi.to_csv('spi_plus.csv')